# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [20]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-65ec1b3dad-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series Without a Trace explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "Without a Trace" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q826477` | Without a Trace    | node |
| `wd:Q733960` | Cold Case       | node |



Also consider

```
wd:Q826477 ?p ?obj .
```

is the BGP to retrieve all **properties of Without a Trace**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should

1. Identify the BGP for television series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for television series

### Get all data property of "Without a Trace" (wd:Q826477)

In [4]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q826477 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1113'), ('label', 'number of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('label', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1267'), ('label', 'AlloCiné series ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('label', 'title')]
[('property', 'http://www.wikidata.org/prop/direct/P1562'), ('label', 'AllMovie title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1712'), ('label', 'Metacritic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2047'), ('label', 'duration')]
[('property', 'http://www.wikidata.org/prop/direct/P2437'), ('label', 'number of seasons')]
[('property', 'http://www.wikidata.org/prop/direct/P2529'), ('label', 'ČSFD film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2603'), ('label', 'Kinopoisk film ID')]
[('property',

### Get all data property of "Without a Trace" (wd:Q826477) that do not contain ID

In [11]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q826477 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1113'), ('label', 'number of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('label', 'title')]
[('property', 'http://www.wikidata.org/prop/direct/P2047'), ('label', 'duration')]
[('property', 'http://www.wikidata.org/prop/direct/P2437'), ('label', 'number of seasons')]
[('property', 'http://www.wikidata.org/prop/direct/P580'), ('label', 'start time')]
[('property', 'http://www.wikidata.org/prop/direct/P582'), ('label', 'end time')]
6


### Get the class of "Without a Trace" (wd:Q826477)

In [5]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q826477 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5398426'), ('label', 'television series')]
1


### Get all object properties of "Without a Trace" (wd:Q826477)

In [6]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q826477 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('label', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('label', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('label', 'creator')]
[('property', 'http://www.wikidata.org/prop/direct/P1811'), ('label', 'list of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P2061'), ('label', 'aspect ratio')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P364'), ('label', 'original language of film or TV show')]
[('property', 'http://www.wikidata.org/prop/direct/P449'), ('label', 'original broadcaster')]
[('property', 'http://www.wikidata.org/prop/direct/P495'), ('label', 'country of origin')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('property', 'http://ww

### Get all the classes connected to "Without a Trace" (wd:Q826477)

In [7]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q826477 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q10689397'), ('label', 'television production company')]
[('objectClass', 'http://www.wikidata.org/entity/Q15961987'), ('label', 'television genre')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q15063611'), ('label', 'city in the state of New York')]
[('objectClass', 'http://www.wikidata.org/entity/Q1093829'), ('label', 'city of the United States')]
[('objectClass', 'http://www.wikidata.org/entity/Q1254874'), ('label', 'television network')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q13406463'), ('label', 'Wikimedia list article')]
[('objectClass', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('objectCl

### Get all the couples (object property - list of the classes of the objects that are connected to "Without a Trace" (wd:Q826477) through that object property)

In [8]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q826477 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2061'), ('labelProperty', 'aspect ratio'), ('objectsClasses', 'http://www.wikidata.org/entity/Q759853,http://www.wikidata.org/entity/Q1441762,http://www.wikidata.org/entity/Q20970434'), ('labelsObjectsClasses', 'film format,aspect ratio,width to height ratio')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('labelProperty', 'creator'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5'), ('labelsObjectsClasses', 'human')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665'), ('labelsObjectsClasses', 'television series season,television series season,television series season,tel

### Get all the couples (object property - list of the  objects that are connected to "Without a Trace" (wd:Q826477) through that object property)

In [9]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q826477 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2061'), ('labelProperty', 'aspect ratio'), ('objects', 'http://www.wikidata.org/entity/Q1383069'), ('labelsObjects', '16:9 aspect ratio')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('labelProperty', 'creator'), ('objects', 'http://www.wikidata.org/entity/Q3126805'), ('labelsObjects', 'Hank Steinberg')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q1120248,http://www.wikidata.org/entity/Q3729810,http://www.wikidata.org/entity/Q3729811,http://www.wikidata.org/entity/Q3729812,http://www.wikidata.org/entity/Q3729815,http://www.wikidata.org/entity/Q3729816,http://www.wikidata.org/entity/Q3729817'), ('labelsObjects', 'Without a Trace, season 1,Without a Trace, season 4,Without a Trace, season 5,Without a Trace, season 2,Without a Trace, season 6,Without a Trace, season 7,Without a Trace, season 3')]
[('property', 'http://www.wik

### Final Query

Final query for this task

In [19]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q826477 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5398426'), ('label', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

### Get all data property of "Without a Trace, season 1" (wd:Q1120248) 

In [5]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1120248 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1113'), ('label', 'number of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('label', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1712'), ('label', 'Metacritic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2529'), ('label', 'ČSFD film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2638'), ('label', 'TV.com ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2704'), ('label', 'EIDR content ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3302'), ('label', 'Open Media Database film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
[('property', 'http://www.wikidata.org/prop/direct/P6381'), ('label', 'iTunes TV season ID')]
10


### Get all data property of "Without a Trace, season 1"  (wd:Q1120248) that do not contain ID

In [6]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1120248 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1113'), ('label', 'number of episodes')]
1


### Get the class of "Without a Trace, season 1" (wd:Q1120248)

In [7]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q1120248 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3464665'), ('label', 'television series season')]
1


### Get all object properties of "Without a Trace, season 1"  (wd:Q1120248)

In [8]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1120248 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P179'), ('label', 'part of the series')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P364'), ('label', 'original language of film or TV show')]
[('property', 'http://www.wikidata.org/prop/direct/P437'), ('label', 'distribution format')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
5


### Get all the classes connected to "Without a Trace, season 1"  (wd:Q1120248)

In [9]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q1120248 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1668024'), ('label', 'service on internet')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q21191270'), ('label', 'television series episode')]
[('objectClass', 'http://www.wikidata.org/entity/Q33742'), ('label', 'natural language')]
[('objectClass', 'http://www.wikidata.org/entity/Q34770'), ('label', 'language')]
[('objectClass', 'http://www.wikidata.org/entity/Q5398426'), ('label', 'television series')]
[('objectClass', 'http://www.wikidata.org/entity/Q81941037'), ('label', 'product distribution method')]
7


### Get all the couples (object property - list of the classes of the objects that are connected to "Without a Trace, season 1"  (wd:Q1120248) through that object property)

In [10]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q1120248 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.org/entity/Q21191270,http://www.wikidata.o

### Get all the couples (object property - list of the  objects that are connected to "Without a Trace, season 1"  (wd:Q1120248) through that object property)

In [11]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q1120248 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q52667904,http://www.wikidata.org/entity/Q52667908,http://www.wikidata.org/entity/Q52667910,http://www.wikidata.org/entity/Q39069555,http://www.wikidata.org/entity/Q52667878,http://www.wikidata.org/entity/Q52667894,http://www.wikidata.org/entity/Q52667898,http://www.wikidata.org/entity/Q52667900,http://www.wikidata.org/entity/Q52667911,http://www.wikidata.org/entity/Q52667916,http://www.wikidata.org/entity/Q52667920,http://www.wikidata.org/entity/Q52667923,http://www.wikidata.org/entity/Q52667926,http://www.wikidata.org/entity/Q52667929,http://www.wikidata.org/entity/Q52667889,http://www.wikidata.org/entity/Q52667891,http://www.wikidata.org/entity/Q52667902,http://www.wikidata.org/entity/Q52667934,http://www.wikidata.org/entity/Q52667939,http://www.wikidata.org/entity/Q52667942,http://www.wikidata.org/entity/Q52667881,http://www.wikidata.org/enti

### Get the value of the data propriety "number of episodes" (wdt:P1113) of "Without a Trace, season 1" (wd:Q1120248)

In [2]:
queryString = """
SELECT DISTINCT ?numberOfEpisodes
WHERE {
   wd:Q1120248 wdt:P1113 ?numberOfEpisodes .
   wd:Q1120248 sc:name ?name .
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('numberOfEpisodes', '23')]
1


Final query for this task

### Final (are 2 queries) 
This task in my opinion can be interpreted in 2 different ways :
- "television series" mentioned in the title of the task can be interpreted generally.
- "television series" is just "Without a Trace" (wd:Q826477)

#### First Interpretation

In [12]:
queryString = """
SELECT DISTINCT ?tvSerie ?labelTvSerie ?numberOfSeasons
WHERE {   
   ?tvSerie wdt:P31 wd:Q5398426 ;
            wdt:P2437 ?numberOfSeasons ;
            sc:name ?labelTvSerie .              
}LIMIT 25
""" 
print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q33323'), ('labelTvSerie', 'Everybody Hates Chris'), ('numberOfSeasons', '4')]
[('tvSerie', 'http://www.wikidata.org/entity/Q500769'), ('labelTvSerie', 'Freaks and Geeks'), ('numberOfSeasons', '1')]
[('tvSerie', 'http://www.wikidata.org/entity/Q1145054'), ('labelTvSerie', 'The NBC Mystery Movie'), ('numberOfSeasons', '6')]
[('tvSerie', 'http://www.wikidata.org/entity/Q344081'), ('labelTvSerie', 'Recess'), ('numberOfSeasons', '3')]
[('tvSerie', 'http://www.wikidata.org/entity/Q708661'), ('labelTvSerie', 'Lindenstraße'), ('numberOfSeasons', '1')]
[('tvSerie', 'http://www.wikidata.org/entity/Q15099450'), ('labelTvSerie', 'Prime Minister and I'), ('numberOfSeasons', '1')]
[('tvSerie', 'http://www.wikidata.org/entity/Q548335'), ('labelTvSerie', 'Angelina Ballerina'), ('numberOfSeasons', '2')]
[('tvSerie', 'http://www.wikidata.org/entity/Q15594658'), ('labelTvSerie', 'Cutthroat Kitchen'), ('numberOfSeasons', '10')]
[('tvSerie', 'http://www

In [13]:
queryString = """
SELECT DISTINCT ?seson ?labelSeason ?numberOfEpisodes
WHERE {   
   ?tvSerie wdt:P31 wd:Q5398426 ;
            wdt:P527 ?season .
   ?seson wdt:P1113 ?numberOfEpisodes ;
          sc:name ?labelSeason .
}LIMIT 25
""" 
print("Results")
x=run_query(queryString)

Results
[('seson', 'http://www.wikidata.org/entity/Q1114794'), ('labelSeason', "Grey's Anatomy, season 1"), ('numberOfEpisodes', '9')]
[('seson', 'http://www.wikidata.org/entity/Q300666'), ('labelSeason', 'Aaahh!!! Real Monsters'), ('numberOfEpisodes', '52')]
[('seson', 'http://www.wikidata.org/entity/Q716172'), ('labelSeason', 'Mobile Fighter G Gundam'), ('numberOfEpisodes', '49')]
[('seson', 'http://www.wikidata.org/entity/Q1890400'), ('labelSeason', "Terry Pratchett's The Colour of Magic"), ('numberOfEpisodes', '2')]
[('seson', 'http://www.wikidata.org/entity/Q33323'), ('labelSeason', 'Everybody Hates Chris'), ('numberOfEpisodes', '88')]
[('seson', 'http://www.wikidata.org/entity/Q500769'), ('labelSeason', 'Freaks and Geeks'), ('numberOfEpisodes', '18')]
[('seson', 'http://www.wikidata.org/entity/Q8350955'), ('labelSeason', 'Cordel Encantado'), ('numberOfEpisodes', '143')]
[('seson', 'http://www.wikidata.org/entity/Q930562'), ('labelSeason', 'Dirty Pair'), ('numberOfEpisodes', '26')

#### Second Interpretation

In [2]:
queryString = """
SELECT DISTINCT ?tvSerie ?labelTvSerie ?numberOfSeasons
WHERE {   
  wd:Q826477 wdt:P31 wd:Q5398426 ;
            wdt:P2437 ?numberOfSeasons ;
            sc:name ?labelTvSerie .              
}LIMIT 25
""" 
print("Results")
x=run_query(queryString)

Results
[('labelTvSerie', 'Without a Trace'), ('numberOfSeasons', '7')]
1


In [3]:
queryString = """
SELECT DISTINCT ?seson ?labelSeason ?numberOfEpisodes
WHERE {   
    wd:Q826477 wdt:P31 wd:Q5398426 ;
            wdt:P527 ?season .
   ?seson wdt:P1113 ?numberOfEpisodes ;
          sc:name ?labelSeason .
}LIMIT 25
""" 
print("Results")
x=run_query(queryString)

Results
[('seson', 'http://www.wikidata.org/entity/Q1114794'), ('labelSeason', "Grey's Anatomy, season 1"), ('numberOfEpisodes', '9')]
[('seson', 'http://www.wikidata.org/entity/Q300666'), ('labelSeason', 'Aaahh!!! Real Monsters'), ('numberOfEpisodes', '52')]
[('seson', 'http://www.wikidata.org/entity/Q716172'), ('labelSeason', 'Mobile Fighter G Gundam'), ('numberOfEpisodes', '49')]
[('seson', 'http://www.wikidata.org/entity/Q1890400'), ('labelSeason', "Terry Pratchett's The Colour of Magic"), ('numberOfEpisodes', '2')]
[('seson', 'http://www.wikidata.org/entity/Q33323'), ('labelSeason', 'Everybody Hates Chris'), ('numberOfEpisodes', '88')]
[('seson', 'http://www.wikidata.org/entity/Q500769'), ('labelSeason', 'Freaks and Geeks'), ('numberOfEpisodes', '18')]
[('seson', 'http://www.wikidata.org/entity/Q8350955'), ('labelSeason', 'Cordel Encantado'), ('numberOfEpisodes', '143')]
[('seson', 'http://www.wikidata.org/entity/Q930562'), ('labelSeason', 'Dirty Pair'), ('numberOfEpisodes', '26')

## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

### Get all data property of "Anthony LaPaglia" (wd:Q308124)

In [20]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q308124 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('label', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1207'), ('label', 'NUKAT ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1220'), ('label', 'Internet Broadway Database person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1263'), ('label', 'NNDB people ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1266'), ('label', 'AlloCiné person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1695'), ('label', 'NLP ID (unique)')]
[('property', 'http://www.wikidata.org/prop/direct/P2019'), ('label', 'AllMovie person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2031'), ('label', 'work period (start)')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2163'), ('label', 

### Get all data property of "Anthony LaPaglia" (wd:Q308124) that do not contain ID

In [19]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q308124 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2031'), ('label', 'work period (start)')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P569'), ('label', 'date of birth')]
4


### Get the class of "Anthony LaPaglia" (wd:Q308124)

In [18]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q308124 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
1


### Get all object properties of "Anthony LaPaglia" (wd:Q308124)

In [17]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q308124 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('label', 'languages spoken, written or signed')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('label', 'award received')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P19'), ('label', 'place of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('label', 'sex or gender')]
[('property', 'http://www.wikidata.org/prop/direct/P26'), ('label', 'spouse')]
[('property', 'http://www.wikidata.org/prop/direct/P27'), ('label', 'country of citizenship')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P3373'), ('label', 'sibling')]
[('property', 'http://www

### Get all the classes connected to "Anthony LaPaglia" (wd:Q308124)

In [16]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q308124 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q186165'), ('label', 'web portal')]
[('objectClass', 'http://www.wikidata.org/entity/Q13218391'), ('label', 'charter city')]
[('objectClass', 'http://www.wikidata.org/entity/Q4220920'), ('label', 'filmmaking occupation')]
[('objectClass', 'http://www.wikidata.org/entity/Q48264'), ('label', 'gender identity')]
[('objectClass', 'http://www.wikidata.org/entity/Q1093829'), ('label', 'city of the United States')]
[('objectClass', 'http://www.wikidata.org/entity/Q123737'), ('label', 'Emmy Award')]
[('objectClass', 'http://www.wikidata.org/entity/Q12737077'), ('label', 'occupation')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q1549591'), ('label', 'big city')]
[('objectClass', 'http://www.wikidata.org/entity/Q1637706'), ('label', 'million city')]
[('objectClass', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('objec

### Get all the couples (object property - list of the classes of the objects that are connected to "Anthony LaPaglia" (wd:Q308124) through that object property)

In [15]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q308124 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12308941'), ('labelsObjectsClasses', 'male given name')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objectsClasses', 'http://www.wikidata.org/entity/Q4220920,http://www.wikidata.org/entity/Q4220920,http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q108300140,http://www.wikidata.org/entity/Q15839299'), ('labelsObjectsClasses', 'filmmaking occupation,filmmaking occupation,occupation,profession,profession,profession,occupation group according to ISCO,theatrical occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('labelProperty', 'sex or gender'), ('objectsClasses', 'http://www.wikidata.org/entity/Q48264,http://www.wikidata.org/entity/Q

### Get all the couples (object property - list of the  objects that are connected to "Anthony LaPaglia" (wd:Q308124) through that object property)

In [14]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q308124 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objects', 'http://www.wikidata.org/entity/Q12241622'), ('labelsObjects', 'Anthony')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objects', 'http://www.wikidata.org/entity/Q10798782,http://www.wikidata.org/entity/Q10800557,http://www.wikidata.org/entity/Q2259451,http://www.wikidata.org/entity/Q3282637,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q937857'), ('labelsObjects', 'television actor,film actor,stage actor,film producer,actor,association football player')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('labelProperty', 'sex or gender'), ('objects', 'http://www.wikidata.org/entity/Q6581097'), ('labelsObjects', 'male')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('labelProperty', 'languages spoken, written or signed'), ('objects', 'http://www.wikidata.org/entity/Q1860'), ('labels

### Get all data property of "Maple Street" (wd:Q52667904)

In [21]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q52667904 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('label', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('label', 'title')]
[('property', 'http://www.wikidata.org/prop/direct/P1562'), ('label', 'AllMovie title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2529'), ('label', 'ČSFD film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2638'), ('label', 'TV.com ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2704'), ('label', 'EIDR content ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3302'), ('label', 'Open Media Database film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P345'), ('label', 'IMDb ID')]
[('property', 'http://www.wikidata.org/prop/direct/P4665'), ('label', 'CineMagia title ID')]
9


### Get all data property of "Maple Street"  (wd:Q52667904) that do not contain ID

In [22]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q52667904 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('label', 'title')]
1


### Get the class of "Maple Street" (wd:Q52667904)

In [23]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q52667904 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q21191270'), ('label', 'television series episode')]
1


### Get all object properties of "Maple Street"  (wd:Q52667904)

In [24]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q52667904 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('label', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P156'), ('label', 'followed by')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('label', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P179'), ('label', 'part of the series')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P364'), ('label', 'original language of film or TV show')]
[('property', 'http://www.wikidata.org/prop/direct/P4908'), ('label', 'season')]
7


### Get all the classes connected to "Maple Street"  (wd:Q52667904)

In [25]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q52667904 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q21191270'), ('label', 'television series episode')]
[('objectClass', 'http://www.wikidata.org/entity/Q33742'), ('label', 'natural language')]
[('objectClass', 'http://www.wikidata.org/entity/Q34770'), ('label', 'language')]
[('objectClass', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
[('objectClass', 'http://www.wikidata.org/entity/Q5398426'), ('label', 'television series')]
[('objectClass', 'http://www.wikidata.org/entity/Q3464665'), ('label', 'television series season')]
7


### Get all the couples (object property - list of the classes of the objects that are connected to "Maple Street"  (wd:Q52667904) through that object property)

In [26]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q52667904 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('labelProperty', 'follows'), ('objectsClasses', 'http://www.wikidata.org/entity/Q21191270'), ('labelsObjectsClasses', 'television series episode')]
[('property', 'http://www.wikidata.org/prop/direct/P364'), ('labelProperty', 'original language of film or TV show'), ('objectsClasses', 'http://www.wikidata.org/entity/Q1288568,http://www.wikidata.org/entity/Q33742,http://www.wikidata.org/entity/Q34770'), ('labelsObjectsClasses', 'modern language,natural language,language')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('labelProperty', 'cast member'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5'), ('labelsObjectsClasses', 'human,human,human,human,human')]
[('property', 'http://www.wikidata.org/prop/direct/P156'), ('labelProperty', 'followed by'), ('objectsClasses',

### Get all the couples (object property - list of the  objects that are connected to "Maple Street"  (wd:Q52667904) through that object property)

In [27]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q52667904 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('labelProperty', 'follows'), ('objects', 'http://www.wikidata.org/entity/Q52667902'), ('labelsObjects', 'Midnight Sun')]
[('property', 'http://www.wikidata.org/prop/direct/P364'), ('labelProperty', 'original language of film or TV show'), ('objects', 'http://www.wikidata.org/entity/Q1860'), ('labelsObjects', 'English')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('labelProperty', 'cast member'), ('objects', 'http://www.wikidata.org/entity/Q430872,http://www.wikidata.org/entity/Q308124,http://www.wikidata.org/entity/Q235075,http://www.wikidata.org/entity/Q551608,http://www.wikidata.org/entity/Q503040'), ('labelsObjects', 'Marianne Jean-Baptiste,Anthony LaPaglia,Poppy Montgomery,Enrique Murciano,Eric Close')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('labelProperty', 'instance of'), ('objects', 'http://www.wikidata.org/entity/Q21191270'), ('labelsObjects', 'television series episode')]
[(

### Get all the objects that contain exactly as label (sc:name) "actor" or "actress" for the object connected through "occupation" (wdt:P106)

In [40]:
queryString = """
SELECT DISTINCT ?occupation ?labelOccupation ?occupationClass ?occupationClassLabel
WHERE 
{   
   ?actor wdt:P106 ?occupation .
   ?occupation sc:name ?labelOccupation ;
               wdt:P31 ?occupationClass .
   ?occupationClass sc:name ?occupationClassLabel .   
   FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  .   
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q8380347'), ('labelOccupation', 'AV actor'), ('occupationClass', 'http://www.wikidata.org/entity/Q12737077'), ('occupationClassLabel', 'occupation')]
[('occupation', 'http://www.wikidata.org/entity/Q26262117'), ('labelOccupation', 'silent film actor'), ('occupationClass', 'http://www.wikidata.org/entity/Q4220920'), ('occupationClassLabel', 'filmmaking occupation')]
[('occupation', 'http://www.wikidata.org/entity/Q52688389'), ('labelOccupation', 'animal actor'), ('occupationClass', 'http://www.wikidata.org/entity/Q1914636'), ('occupationClassLabel', 'activity')]
[('occupation', 'http://www.wikidata.org/entity/Q10798782'), ('labelOccupation', 'television actor'), ('occupationClass', 'http://www.wikidata.org/entity/Q28640'), ('occupationClassLabel', 'profession')]
[('occupation', 'http://www.wikidata.org/entity/Q10800557'), ('labelOccupation', 'film actor'), ('occupationClass', 'http://www.wikidata.org/entity/Q4220920'), ('occupation

### Final Query
This task in my opinion can be interpreted in 2 different ways :

"television series episodes" mentioned in the text the task can be interpreted generally.
"television series episodes" are the episodes of "Without a Trace" (wd:Q826477)

Final query for this task

#### First Interpretation

In [22]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember (COUNT(?tvSerieEpisode) AS ?numberOfEpisodes)
WHERE 
{   
   #Among all episodes of "television series" select the cast members that are actors or actress and, count the number of episodes such actors have done
   ?tvSerieEpisode wdt:P31 wd:Q21191270 ;
                   wdt:P161 ?castMember .
   ?castMember wdt:P106 ?occupation .   
   
   #Labels
   ?occupation sc:name ?labelOccupation .
   ?castMember sc:name ?labelCastMember .
   
   FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  .   
}GROUP BY ?castMember ?labelCastMember
ORDER BY DESC (?numberOfEpisodes)
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q32522'), ('labelCastMember', 'Jennifer Aniston'), ('numberOfEpisodes', '1185')]
[('castMember', 'http://www.wikidata.org/entity/Q179041'), ('labelCastMember', 'Lisa Kudrow'), ('numberOfEpisodes', '1180')]
[('castMember', 'http://www.wikidata.org/entity/Q316955'), ('labelCastMember', 'Michael Dorn'), ('numberOfEpisodes', '1004')]
[('castMember', 'http://www.wikidata.org/entity/Q58912'), ('labelCastMember', 'Courteney Cox'), ('numberOfEpisodes', '960')]
[('castMember', 'http://www.wikidata.org/entity/Q16296'), ('labelCastMember', 'Patrick Stewart'), ('numberOfEpisodes', '950')]
[('castMember', 'http://www.wikidata.org/entity/Q188792'), ('labelCastMember', 'David Schwimmer'), ('numberOfEpisodes', '944')]
[('castMember', 'http://www.wikidata.org/entity/Q311453'), ('labelCastMember', 'Brent Spiner'), ('numberOfEpisodes', '925')]
[('castMember', 'http://www.wikidata.org/entity/Q103946'), ('labelCastMember', 'Kiefer Sutherland'), ('numb

#### Second Interpretation

In [21]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember (COUNT(?tvSerieEpisode) AS ?numberOfEpisodes)
WHERE 
{   
   #Among all episodes of "television series" select the cast members that are actors or actress and, count the number of episodes such actors have done
   wd:Q826477 wdt:P527 ?tvSeriePart .
   ?tvSeriePart wdt:P527 ?tvSerieEpisode .
   ?tvSerieEpisode wdt:P31 wd:Q21191270 ;
                   wdt:P161 ?castMember .
   ?castMember wdt:P106 ?occupation .      
   
   #Labels
   ?occupation sc:name ?labelOccupation .
   ?castMember sc:name ?labelCastMember .
   
   FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  .   
}GROUP BY ?castMember ?labelCastMember 
ORDER BY DESC (?numberOfEpisodes)
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('labelCastMember', 'Anthony LaPaglia'), ('numberOfEpisodes', '640')]
[('castMember', 'http://www.wikidata.org/entity/Q503040'), ('labelCastMember', 'Eric Close'), ('numberOfEpisodes', '640')]
[('castMember', 'http://www.wikidata.org/entity/Q235075'), ('labelCastMember', 'Poppy Montgomery'), ('numberOfEpisodes', '480')]
[('castMember', 'http://www.wikidata.org/entity/Q551608'), ('labelCastMember', 'Enrique Murciano'), ('numberOfEpisodes', '480')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('labelCastMember', 'Marianne Jean-Baptiste'), ('numberOfEpisodes', '480')]
[('castMember', 'http://www.wikidata.org/entity/Q373989'), ('labelCastMember', 'David Paymer'), ('numberOfEpisodes', '5')]
[('castMember', 'http://www.wikidata.org/entity/Q706513'), ('labelCastMember', 'Charles S. Dutton'), ('numberOfEpisodes', '4')]
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('labelCastMember', 'Bruce Davison'),

## Task 4
Check who is the actor who acted in more films while working on "Without a Trace" (the result set must be a list of triples actor/actress IRI, label and #films).

### Get all data property of "Cold Case" (wd:Q733960)

In [44]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q733960 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P5829'), ('label', 'Television Academy Foundation show ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1113'), ('label', 'number of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('label', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1267'), ('label', 'AlloCiné series ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('label', 'title')]
[('property', 'http://www.wikidata.org/prop/direct/P1712'), ('label', 'Metacritic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2013'), ('label', 'Facebook ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2047'), ('label', 'duration')]
[('property', 'http://www.wikidata.org/prop/direct/P2437'), ('label', 'number of seasons')]
[('property', 'http://www.wikidata.org/prop/direct/P2529'), ('label', 'ČSFD film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]

### Get all data property of "Cold Case" (wd:Q733960) that do not contain ID

In [45]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q733960 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1113'), ('label', 'number of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('label', 'title')]
[('property', 'http://www.wikidata.org/prop/direct/P2047'), ('label', 'duration')]
[('property', 'http://www.wikidata.org/prop/direct/P2437'), ('label', 'number of seasons')]
[('property', 'http://www.wikidata.org/prop/direct/P580'), ('label', 'start time')]
[('property', 'http://www.wikidata.org/prop/direct/P582'), ('label', 'end time')]
[('property', 'http://www.wikidata.org/prop/direct/P8278'), ('label', 'serial identifier on SerialZone')]
7


### Get the class of "Cold Case" (wd:Q733960)

In [47]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q733960 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5398426'), ('label', 'television series')]
1


### Get all object properties of "Cold Case" (wd:Q733960)

In [48]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q733960 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('label', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('label', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('label', 'creator')]
[('property', 'http://www.wikidata.org/prop/direct/P1811'), ('label', 'list of episodes')]
[('property', 'http://www.wikidata.org/prop/direct/P2061'), ('label', 'aspect ratio')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P364'), ('label', 'original language of film or TV show')]
[('property', 'http://www.wikidata.org/prop/direct/P449'), ('label', 'original broadcaster')]
[('property', 'http://www.wikidata.org/prop/direct/P495'), ('label', 'country of origin')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('property', 'http://ww

### Get all the classes connected to "Cold Case" (wd:Q733960)

In [49]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q733960 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q95074'), ('label', 'fictional character')]
[('objectClass', 'http://www.wikidata.org/entity/Q15961987'), ('label', 'television genre')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q1758690'), ('label', 'municipal police')]
[('objectClass', 'http://www.wikidata.org/entity/Q21010817'), ('label', 'city of Pennsylvania')]
[('objectClass', 'http://www.wikidata.org/entity/Q1254874'), ('label', 'television network')]
[('objectClass', 'http://www.wikidata.org/entity/Q1288568'), ('label', 'modern language')]
[('objectClass', 'http://www.wikidata.org/entity/Q13406463'), ('label', 'Wikimedia list article')]
[('objectClass', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('objectClass', 'http://www.wikidata.org/

### Get all the couples (object property - list of the classes of the objects that are connected to "Cold Case" (wd:Q733960) through that object property)

In [50]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q733960 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2061'), ('labelProperty', 'aspect ratio'), ('objectsClasses', 'http://www.wikidata.org/entity/Q759853,http://www.wikidata.org/entity/Q1441762,http://www.wikidata.org/entity/Q20970434'), ('labelsObjectsClasses', 'film format,aspect ratio,width to height ratio')]
[('property', 'http://www.wikidata.org/prop/direct/P57'), ('labelProperty', 'director'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5'), ('labelsObjectsClasses', 'human')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('labelProperty', 'creator'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5'), ('labelsObjectsClasses', 'human')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http://www.wikidata.org/entity/Q3464665,http:/

### Get all the couples (object property - list of the  objects that are connected to "Cold Case" (wd:Q733960) through that object property)

In [14]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q733960 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2061'), ('labelProperty', 'aspect ratio'), ('objects', 'http://www.wikidata.org/entity/Q1383069'), ('labelsObjects', '16:9 aspect ratio')]
[('property', 'http://www.wikidata.org/prop/direct/P57'), ('labelProperty', 'director'), ('objects', 'http://www.wikidata.org/entity/Q217037'), ('labelsObjects', 'Jerry Bruckheimer')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('labelProperty', 'creator'), ('objects', 'http://www.wikidata.org/entity/Q545377'), ('labelsObjects', 'Meredith Stiehm')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q671337,http://www.wikidata.org/entity/Q517863,http://www.wikidata.org/entity/Q717713,http://www.wikidata.org/entity/Q834361,http://www.wikidata.org/entity/Q987172,http://www.wikidata.org/entity/Q987173,http://www.wikidata.org/entity/Q987176'), ('labelsObjects', 'Cold Case, season 6,Cold Case, season

### Get the superclass of "television series" wd:Q5398426

In [22]:
queryString = """
SELECT DISTINCT ?superClass ?labelSuperClass
WHERE 
{   
    wd:Q5398426 wdt:P279 ?superClass .
    
    ?superClass sc:name ?labelSuperClass .
}
"""
print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q15416'), ('labelSuperClass', 'television program')]
[('superClass', 'http://www.wikidata.org/entity/Q7725310'), ('labelSuperClass', 'series of creative works')]
2


### Get the superclass of "series of creative works" wd:Q7725310

In [23]:
queryString = """
SELECT DISTINCT ?superClass ?labelSuperClass
WHERE 
{   
    wd:Q7725310 wdt:P279 ?superClass .
    
    ?superClass sc:name ?labelSuperClass .
}
"""
print("Results")
x=run_query(queryString)

Results
[('superClass', 'http://www.wikidata.org/entity/Q20937557'), ('labelSuperClass', 'series')]
[('superClass', 'http://www.wikidata.org/entity/Q17489659'), ('labelSuperClass', 'group of works')]
2


### Get all subclasses of "series" wd:Q20937557

In [24]:
queryString = """
SELECT DISTINCT ?subClass ?labelSubclass
WHERE 
{   
    ?subClass wdt:P279 wd:Q20937557 ;
              sc:name ?labelSubclass .
}
"""
print("Results")
x=run_query(queryString)

Results
[('subClass', 'http://www.wikidata.org/entity/Q332225'), ('labelSubclass', 'endgame')]
[('subClass', 'http://www.wikidata.org/entity/Q98391050'), ('labelSubclass', 'nuclear tests series')]
[('subClass', 'http://www.wikidata.org/entity/Q31836768'), ('labelSubclass', 'model series')]
[('subClass', 'http://www.wikidata.org/entity/Q133250'), ('labelSubclass', 'sequence')]
[('subClass', 'http://www.wikidata.org/entity/Q3249551'), ('labelSubclass', 'process')]
[('subClass', 'http://www.wikidata.org/entity/Q11471'), ('labelSubclass', 'time')]
[('subClass', 'http://www.wikidata.org/entity/Q52905625'), ('labelSubclass', 'numismatic serie')]
[('subClass', 'http://www.wikidata.org/entity/Q12538685'), ('labelSubclass', 'roadshow')]
[('subClass', 'http://www.wikidata.org/entity/Q11424'), ('labelSubclass', 'film')]
[('subClass', 'http://www.wikidata.org/entity/Q15142889'), ('labelSubclass', 'weapon family')]
[('subClass', 'http://www.wikidata.org/entity/Q1914636'), ('labelSubclass', 'activit

### Get all subclasses of "series" wd:Q20937557 that have the label that starts with "f"

In [25]:
queryString = """
SELECT DISTINCT ?subClass ?labelSubclass
WHERE 
{   
    ?subClass wdt:P279 wd:Q20937557 ;
              sc:name ?labelSubclass .
    FILTER REGEX(?labelSubclass, "^f")
}
"""
print("Results")
x=run_query(queryString)

Results
[('subClass', 'http://www.wikidata.org/entity/Q11424'), ('labelSubclass', 'film')]
1


### Get the first 25 instance of (wdt:P31) of "film" (wd:Q11424)

In [27]:
queryString = """
SELECT DISTINCT ?film ?labelFilm
WHERE 
{   
    ?film wdt:P31 wd:Q11424 ;
              sc:name ?labelFilm .
}LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q8178307'), ('labelFilm', '15 Park Avenue')]
[('film', 'http://www.wikidata.org/entity/Q8197668'), ('labelFilm', 'Idiot Love')]
[('film', 'http://www.wikidata.org/entity/Q319263'), ('labelFilm', 'Doctor in the House')]
[('film', 'http://www.wikidata.org/entity/Q8452990'), ('labelFilm', "Sunday League - Pepik Hnatek's Final Match")]
[('film', 'http://www.wikidata.org/entity/Q284184'), ('labelFilm', 'Ra.One')]
[('film', 'http://www.wikidata.org/entity/Q13637926'), ('labelFilm', 'Indra')]
[('film', 'http://www.wikidata.org/entity/Q13638153'), ('labelFilm', 'Inner Senses')]
[('film', 'http://www.wikidata.org/entity/Q13645807'), ('labelFilm', 'Madhumasam')]
[('film', 'http://www.wikidata.org/entity/Q13647258'), ('labelFilm', 'Marupakkam')]
[('film', 'http://www.wikidata.org/entity/Q13677566'), ('labelFilm', 'Timberjack')]
[('film', 'http://www.wikidata.org/entity/Q13693967'), ('labelFilm', 'Cross')]
[('film', 'http://www.wikidata.org/entity/

### Get all data property of "15 Park Avenue" (wd:Q11424)

In [34]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q11424 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1256'), ('label', 'Iconclass notation')]
[('property', 'http://www.wikidata.org/prop/direct/P1368'), ('label', 'LNB ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1617'), ('label', 'BBC Things ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2167'), ('label', 'UNSPSC Code')]
[('property', 'http://www.wikidata.org

### Get all data property of "15 Park Avenue"  (wd:Q11424) that do not contain ID

In [33]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q11424 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1256'), ('label', 'Iconclass notation')]
[('property', 'http://www.wikidata.org/prop/direct/P2167'), ('label', 'UNSPSC Code')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P3921'), ('label', 'Wikidata SPARQL query equivalent')]
[('property', 'http://www.wikidata.org/prop/direct/P3984'), ('label', 'subreddit')]
[('property', 'http://www.wikidata.org/prop/direct/P5250'), ('label', 'IAB code')]
[('property', 'http://www.wikidata.org/prop/direct/P7497'), ('label', 'Wolfram Language entity type')]
[('property', 'http://www.wikidata.

### Get the class of "15 Park Avenue" (wd:Q11424)

In [32]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q11424 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get all object properties of "15 Park Avenue"  (wd:Q11424)

In [31]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q11424 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P2579'), ('label', 'studied by')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P2888'), ('label', 'exact match')]
[('property', 'http://www.wikidata.org/prop/direct/P2959'), ('label', 'permanent dupl

### Get all the classes connected to "15 Park Avenue"  (wd:Q11424)

In [30]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q11424 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q27659414'), ('label', 'Wikidata property related to fashion')]
[('objectClass', 'http://www.wikidata.org/entity/Q46275839'), ('label', 'Wikidata property related to theater')]
[('objectClass', 'http://www.wikidata.org/entity/Q107395292'), ('label', 'Wikidata property related to movies and television shows')]
[('objectClass', 'http://www.wikidata.org/entity/Q66116790'), ('label', 'Wikidata property to identify music videos')]
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q19307174'), ('label', 'digital distribution platform')]
[('objectClass', 'http://www.wikidata.org/entity/Q28147643'), ('label', 'Wikidata property related to video games')]
[('objectClass', 'http://www.wikidata.org/entity/Q29542094'), ('label', 'Wikidata property to identify films')]
[('objectClass', 'http://www.wikidata.org/entity/Q51077473'), ('label', 'Wikidata property

### Get all the couples (object property - list of the classes of the objects that are connected to "15 Park Avenue"  (wd:Q11424) through that object property)

In [29]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q11424 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P5125'), ('labelProperty', 'Wikimedia outline'), ('objectsClasses', 'http://www.wikidata.org/entity/Q13406463,http://www.wikidata.org/entity/Q26884324'), ('labelsObjectsClasses', 'Wikimedia list article,Wikimedia outline article')]
[('property', 'http://www.wikidata.org/prop/direct/P5008'), ('labelProperty', 'on focus list of Wikimedia project'), ('objectsClasses', 'http://www.wikidata.org/entity/Q16695773,http://www.wikidata.org/entity/Q13406463,http://www.wikidata.org/entity/Q14204246'), ('labelsObjectsClasses', 'WikiProject,Wikimedia list article,Wikimedia project page')]
[('property', 'http://www.wikidata.org/prop/direct/P5869'), ('labelProperty', 'model item'), ('objectsClasses', 'http://www.wikidata.org/entity/Q336144,http://www.wikidata.org/entity/Q11424,http://www.wikidata.org/entity/Q11424,http://www.wikidata.org/entity/Q11424,http://www.wikidata.org/entity/Q11424'), ('labelsObjectsClasses', 'anthology film,film,film,f

### Get all the couples (object property - list of the  objects that are connected to "15 Park Avenue"  (wd:Q11424) through that object property)

In [28]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q11424 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P5125'), ('labelProperty', 'Wikimedia outline'), ('objects', 'http://www.wikidata.org/entity/Q7112632'), ('labelsObjects', 'outline of film')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q441606,http://www.wikidata.org/entity/Q20820424,http://www.wikidata.org/entity/Q795160,http://www.wikidata.org/entity/Q622550,http://www.wikidata.org/entity/Q4458345'), ('labelsObjects', 'title sequence,credit,film frame,trailer,motion picture credits')]
[('property', 'http://www.wikidata.org/prop/direct/P5008'), ('labelProperty', 'on focus list of Wikimedia project'), ('objects', 'http://www.wikidata.org/entity/Q5460604'), ('labelsObjects', 'Wikipedia:Vital articles')]
[('property', 'http://www.wikidata.org/prop/direct/P5869'), ('labelProperty', 'model item'), ('objects', 'http://www.wikidata.org/entity/Q17738,http://www.wikidata.org/entity/Q192724,http:

### Get all proprieties that connects "actor/actress" (wdt:P106) with "15 Park Avenue" (wd:Q11424)

In [36]:
queryString = """
SELECT DISTINCT ?propriety ?labelPropriety
WHERE 
{   
    ?actor wdt:P106 ?occupation ;
           ?propriety wd:Q11424 .
           
    #Labels
    ?occupation sc:name ?labelOccupation .   
    ?propriety sc:name ?labelPropriety .
    
    #Occupation must be actor
    FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  . 
}
LIMIT 50 
"""
print("Results")
x=run_query(queryString)

Results
[('propriety', 'http://www.wikidata.org/prop/direct/P101'), ('labelPropriety', 'field of work')]
[('propriety', 'http://www.wikidata.org/prop/direct/P136'), ('labelPropriety', 'genre')]
[('propriety', 'http://www.wikidata.org/prop/direct/P1056'), ('labelPropriety', 'product or material produced')]
[('propriety', 'http://www.wikidata.org/prop/direct/P106'), ('labelPropriety', 'occupation')]
[('propriety', 'http://www.wikidata.org/prop/direct/P135'), ('labelPropriety', 'movement')]
5


### Get first 25 "cast members" of any film

In [44]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember ?film ?labelFilm
WHERE 
{   
    ?film wdt:P161 ?castMember .
    ?film wdt:P31 wd:Q11424 .
    
    #Labels
    ?castMember sc:name ?labelCastMember .
    ?film sc:name ?labelFilm .
}
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q242964'), ('labelCastMember', 'Sophie Evans'), ('film', 'http://www.wikidata.org/entity/Q8197668'), ('labelFilm', 'Idiot Love')]
[('castMember', 'http://www.wikidata.org/entity/Q3818317'), ('labelCastMember', 'Roger Casamajor'), ('film', 'http://www.wikidata.org/entity/Q8197668'), ('labelFilm', 'Idiot Love')]
[('castMember', 'http://www.wikidata.org/entity/Q104878263'), ('labelCastMember', 'Jan Václavík'), ('film', 'http://www.wikidata.org/entity/Q8452990'), ('labelFilm', "Sunday League - Pepik Hnatek's Final Match")]
[('castMember', 'http://www.wikidata.org/entity/Q106107428'), ('labelCastMember', 'Tereza Gübelová'), ('film', 'http://www.wikidata.org/entity/Q8452990'), ('labelFilm', "Sunday League - Pepik Hnatek's Final Match")]
[('castMember', 'http://www.wikidata.org/entity/Q10769440'), ('labelCastMember', 'Alena Mihulová'), ('film', 'http://www.wikidata.org/entity/Q8452990'), ('labelFilm', "Sunday League - Pepik Hnatek's Fina

### Final Query

Final query for this task

In [13]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember (COUNT(DISTINCT ?film) AS ?numberOfFilms) 
WHERE 
{               
    wd:Q826477 wdt:P527 ?season .
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 ?castMember .
    
    
    ?film wdt:P161 ?castMember ;
          wdt:P31 wd:Q11424 .
    
    #Labels
    ?castMember sc:name ?labelCastMember .
          
}GROUP BY ?castMember ?labelCastMember
ORDER BY DESC(?numberOfFilms)
LIMIT 1
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('labelCastMember', 'Bruce Davison'), ('numberOfFilms', '56')]
1


## Task 5
Compare Without a Trace with the tv series "Cold Case" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

### Get "title" (wdt:P1476) "numberOfSeasons" (wdt:P2437) "numberOfEpisodes" (wdt:P1113) of "Without a Trace" (wd:Q826477)

In [72]:
queryString = """
SELECT ?title ?numberOfSeasons ?numberOfEpisodes
WHERE 
{   
    wd:Q826477 wdt:P1476 ?title ;
             wdt:P2437 ?numberOfSeasons ;
             wdt:P1113 ?numberOfEpisodes .
}
LIMIT 50 
"""
print("Results")
x=run_query(queryString)

Results
[('title', 'Without a Trace'), ('numberOfSeasons', '7'), ('numberOfEpisodes', '160')]
1


### Final Query

Final query for this task

In [74]:
queryString = """
SELECT ?tvSerie ?labelTvSerie ?numberOfSeasons ?numberOfEpisodes (COUNT(DISTINCT ?castMember) AS ?numberCastMembers)
WHERE 
{   
    ?tvSerie wdt:P1476 ?title ;
             wdt:P2437 ?numberOfSeasons ;
             wdt:P1113 ?numberOfEpisodes ;
             wdt:P527 ?season ;
             sc:name ?labelTvSerie .             
    ?season wdt:P527 ?episode .
    ?episode wdt:P161 ?castMember .
        
    FILTER (str(?title) = "Without a Trace" || str(?title) = "Cold Case")
}GROUP BY ?tvSerie ?labelTvSerie ?numberOfSeasons ?numberOfEpisodes
LIMIT 50  
"""
print("Results")
x=run_query(queryString)

Results
[('tvSerie', 'http://www.wikidata.org/entity/Q733960'), ('labelTvSerie', 'Cold Case'), ('numberOfSeasons', '7'), ('numberOfEpisodes', '156'), ('numberCastMembers', '674')]
[('tvSerie', 'http://www.wikidata.org/entity/Q826477'), ('labelTvSerie', 'Without a Trace'), ('numberOfSeasons', '7'), ('numberOfEpisodes', '160'), ('numberCastMembers', '30')]
2


## Task 6
Return the actors who are members of the cast of Without a Trace have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

### Get all object proprieties of "John Livingston" (wd:Q18618690)

In [9]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q18618690 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P19'), ('label', 'place of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('label', 'sex or gender')]
[('property', 'http://www.wikidata.org/prop/direct/P27'), ('label', 'country of citizenship')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P734'), ('label', 'family name')]
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('label', 'given name')]
7


### Get "Kevin Bacon" actor BGP

In [83]:
queryString = """
SELECT DISTINCT ?kevinBacon ?labelKevinBacon
WHERE 
{   
    ?kevinBacon wdt:P106 ?occupation ;
           wdt:P735 ?givenName ;
           wdt:P734 ?familyName .    
    
    #Labels
    ?givenName sc:name ?labelGivenName .
    ?familyName sc:name ?labelFamilyName .
    ?occupation sc:name ?labelOccupation .   
    ?kevinBacon sc:name ?labelKevinBacon .
    
    #Occupation must be actor
    FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  . 
    # Name = "Kevin" AND Surname = "Bacon"
    FILTER (CONTAINS(?labelGivenName, "Kevin") && CONTAINS(?labelFamilyName, "Bacon")) .
}
LIMIT 50 
"""
print("Results")
x=run_query(queryString)

Results
[('kevinBacon', 'http://www.wikidata.org/entity/Q3454165'), ('labelKevinBacon', 'Kevin Bacon')]
1


### Get all "cast members" (wdt:P161) of "Without a Trace" (wd:Q826477)

In [10]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember 
WHERE 
{   
    {
        SELECT DISTINCT ?castMember ?labelCastMember
        WHERE
        {
            wd:Q826477 wdt:P527 ?season .
            ?season wdt:P527 ?episode .
            ?episode wdt:P161 ?castMember .
            ?castMember sc:name ?labelCastMember .
        }
    }    
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q373989'), ('labelCastMember', 'David Paymer')]
[('castMember', 'http://www.wikidata.org/entity/Q139341'), ('labelCastMember', 'Zach Grenier')]
[('castMember', 'http://www.wikidata.org/entity/Q3265830'), ('labelCastMember', 'Lucille Soong')]
[('castMember', 'http://www.wikidata.org/entity/Q430872'), ('labelCastMember', 'Marianne Jean-Baptiste')]
[('castMember', 'http://www.wikidata.org/entity/Q238501'), ('labelCastMember', 'Alex Veadov')]
[('castMember', 'http://www.wikidata.org/entity/Q308124'), ('labelCastMember', 'Anthony LaPaglia')]
[('castMember', 'http://www.wikidata.org/entity/Q1721796'), ('labelCastMember', 'Kaitlin Doubleday')]
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('labelCastMember', 'Bruce Davison')]
[('castMember', 'http://www.wikidata.org/entity/Q22704603'), ('labelCastMember', 'Lily Knight')]
[('castMember', 'http://www.wikidata.org/entity/Q445325'), ('labelCastMember', 'Megan Gallagher')]
[('cas

### Get all the "cast members" (wdt:P161) with Kevin Bacon Number equals 2

In [96]:
queryString = """
SELECT ?castMember ?labelCastMember
WHERE
{
    ?castMember (^wdt:P161 / wdt:P161){2} ?kevinBacon .
    ?castMember sc:name ?labelCastMember .
    #Kevin Bacon URI
    {
        SELECT DISTINCT ?kevinBacon ?labelKevinBacon
        WHERE 
        {   
            ?kevinBacon wdt:P106 ?occupation ;
                   wdt:P735 ?givenName ;
                   wdt:P734 ?familyName .    

            #Labels
            ?givenName sc:name ?labelGivenName .
            ?familyName sc:name ?labelFamilyName .
            ?occupation sc:name ?labelOccupation .   
            ?kevinBacon sc:name ?labelKevinBacon .

            #Occupation must be actor
            FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  . 
            # Name = "Kevin" AND Surname = "Bacon"
            FILTER (CONTAINS(?labelGivenName, "Kevin") && CONTAINS(?labelFamilyName, "Bacon")) .
        }LIMIT 1
    }
} LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q5372060'), ('labelCastMember', 'Emily Bett Rickards')]
[('castMember', 'http://www.wikidata.org/entity/Q16185788'), ('labelCastMember', 'Kyle Catlett')]
[('castMember', 'http://www.wikidata.org/entity/Q15058295'), ('labelCastMember', 'Josh Brener')]
[('castMember', 'http://www.wikidata.org/entity/Q266888'), ('labelCastMember', 'Vinessa Shaw')]
[('castMember', 'http://www.wikidata.org/entity/Q21634507'), ('labelCastMember', 'Justin Isfeld')]
[('castMember', 'http://www.wikidata.org/entity/Q22943048'), ('labelCastMember', "Tom O'Rourke")]
[('castMember', 'http://www.wikidata.org/entity/Q237214'), ('labelCastMember', 'Lesley Ann Warren')]
[('castMember', 'http://www.wikidata.org/entity/Q275476'), ('labelCastMember', 'Kay Lenz')]
[('castMember', 'http://www.wikidata.org/entity/Q275476'), ('labelCastMember', 'Kay Lenz')]
[('castMember', 'http://www.wikidata.org/entity/Q275476'), ('labelCastMember', 'Kay Lenz')]
[('castMember', 'http:/

Final query for this task

### Final Query

In [21]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember
WHERE
{
    ?castMember (^wdt:P161 / wdt:P161){2} ?kevinBacon .
    #Kevin Bacon URI
    {
        SELECT DISTINCT ?kevinBacon ?labelKevinBacon
        WHERE 
        {   
            ?kevinBacon wdt:P106 ?occupation ;
                   wdt:P735 ?givenName ;
                   wdt:P734 ?familyName .    

            #Labels
            ?givenName sc:name ?labelGivenName .
            ?familyName sc:name ?labelFamilyName .
            ?occupation sc:name ?labelOccupation .   
            ?kevinBacon sc:name ?labelKevinBacon .

            #Occupation must be actor
            FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  . 
            # Name = "Kevin" AND Surname = "Bacon"
            FILTER (CONTAINS(?labelGivenName, "Kevin") && CONTAINS(?labelFamilyName, "Bacon")) .
        }LIMIT 1
    }    
    #Cast Members of Without a Trace
    {                        
        SELECT DISTINCT ?castMember ?labelCastMember
        WHERE
        {
            wd:Q826477 wdt:P527 ?season .
            ?season wdt:P527 ?episode .
            ?episode wdt:P161 ?castMember .
            ?castMember sc:name ?labelCastMember .
        }LIMIT 30                           
    }       
} LIMIT 5 
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q462343'), ('labelCastMember', 'Rosemary Forsyth')]
[('castMember', 'http://www.wikidata.org/entity/Q18618690'), ('labelCastMember', 'John Livingston')]
[('castMember', 'http://www.wikidata.org/entity/Q1125651'), ('labelCastMember', 'Thom Barry')]
[('castMember', 'http://www.wikidata.org/entity/Q355163'), ('labelCastMember', 'Bruce Davison')]
[('castMember', 'http://www.wikidata.org/entity/Q139341'), ('labelCastMember', 'Zach Grenier')]
5


## Task 7
Consider the actors who are members of the cast of Cold Case. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

### Get all the "cast members" (wdt:P161) that are actors of "Cold Case" (wd:Q733960)

In [8]:
queryString = """
SELECT DISTINCT ?castMember ?labelCastMember 
WHERE 
{   
    #Cast Members of wd:Q733960 
    {
        SELECT DISTINCT ?castMember ?labelCastMember
        WHERE
        {
            wd:Q733960 wdt:P161 ?castMember .
            ?castMember sc:name ?labelCastMember .
        }
    }    
    #Cast Members of seasons of wd:Q733960 
    UNION
    {
        SELECT DISTINCT ?castMember ?labelCastMember
        WHERE
        {
            wd:Q733960 wdt:P527 ?season .
            ?season wdt:P161 ?castMember .
            ?castMember sc:name ?labelCastMember .
        }
    }       
    #Cast Members of episodes of wd:Q733960 
    UNION
    {
        SELECT DISTINCT ?castMember ?labelCastMember
        WHERE
        {
            wd:Q733960 wdt:P527 ?season .
            ?season wdt:P527 ?episode .
            ?episode wdt:P161 ?castMember .
            ?castMember sc:name ?labelCastMember .
        }
    }
    ?castMember wdt:P106 ?occupation .
    ?occupation sc:name ?labelOccupation .    
    FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  .
}
"""
print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q270382'), ('labelCastMember', 'Justina Machado')]
[('castMember', 'http://www.wikidata.org/entity/Q269891'), ('labelCastMember', 'Julianna Guill')]
[('castMember', 'http://www.wikidata.org/entity/Q128821'), ('labelCastMember', 'Stephen Full')]
[('castMember', 'http://www.wikidata.org/entity/Q1340897'), ('labelCastMember', 'Martin Spanjers')]
[('castMember', 'http://www.wikidata.org/entity/Q275476'), ('labelCastMember', 'Kay Lenz')]
[('castMember', 'http://www.wikidata.org/entity/Q441617'), ('labelCastMember', 'Gabrielle Miller')]
[('castMember', 'http://www.wikidata.org/entity/Q613378'), ('labelCastMember', 'Bruno Campos')]
[('castMember', 'http://www.wikidata.org/entity/Q242206'), ('labelCastMember', 'Bree Turner')]
[('castMember', 'http://www.wikidata.org/entity/Q258246'), ('labelCastMember', 'Lois Smith')]
[('castMember', 'http://www.wikidata.org/entity/Q261926'), ('labelCastMember', 'Muse Watson')]
[('castMember', 'http://www

### Get all the "television series" such that the "cast members" (wdt:P161) of "Cold Case" are cast members

In [15]:
queryString = """
SELECT DISTINCT ?televisionSerie ?labelTelevisionSerie
{
    ?televisionSerie wdt:P161 ?castMember ;
                     wdt:P31 wd:Q5398426 ;
                     sc:name ?labelTelevisionSerie .
    
    
    #Cast Members of Cold Case
    {
        SELECT DISTINCT ?castMember ?labelCastMember 
        WHERE 
        {   
            #TODO DO THE SAME ON QUERY N° 4
            #Cast Members of wd:Q733960 
            {
                SELECT DISTINCT ?castMember ?labelCastMember
                WHERE
                {
                    wd:Q733960 wdt:P161 ?castMember .
                    ?castMember sc:name ?labelCastMember .
                }
            }    
            #Cast Members of seasons of wd:Q733960 
            UNION
            {
                SELECT DISTINCT ?castMember ?labelCastMember
                WHERE
                {
                    wd:Q733960 wdt:P527 ?season .
                    ?season wdt:P161 ?castMember .
                    ?castMember sc:name ?labelCastMember .
                }
            }       
            #Cast Members of episodes of wd:Q733960 
            UNION
            {
                SELECT DISTINCT ?castMember ?labelCastMember
                WHERE
                {
                    wd:Q733960 wdt:P527 ?season .
                    ?season wdt:P527 ?episode .
                    ?episode wdt:P161 ?castMember .
                    ?castMember sc:name ?labelCastMember .
                }
            }
            ?castMember wdt:P106 ?occupation .
            ?occupation sc:name ?labelOccupation .    
            FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  .
        }
    }
}
"""
print("Results")
x=run_query(queryString)

Results
[('televisionSerie', 'http://www.wikidata.org/entity/Q17049828'), ('labelTelevisionSerie', 'Bad Samaritans')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q17635847'), ('labelTelevisionSerie', 'K.C. Undercover')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q221230'), ('labelTelevisionSerie', 'Nip/Tuck')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q2947967'), ('labelTelevisionSerie', 'Under the Dome')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q23558'), ('labelTelevisionSerie', 'House')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q498569'), ('labelTelevisionSerie', 'Rules of Engagement')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q1190808'), ('labelTelevisionSerie', 'Mental')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q186219'), ('labelTelevisionSerie', 'True Blood')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q589187'), ('labelTelevisionSerie', 'Sliders')]
[('televisionSerie', 'http://www.wikidata.o

### Get all the couples (object property - list of the classes of the objects that are connected to "Meredith Stiehm"  (wd:Q545377) through that object property)

In [12]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q545377 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objectsClasses', 'http://www.wikidata.org/entity/Q3409032'), ('labelsObjectsClasses', 'unisex given name')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objectsClasses', 'http://www.wikidata.org/entity/Q4220920,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q4164871'), ('labelsObjectsClasses', 'filmmaking occupation,profession,position')]
[('property', 'http://www.wikidata.org/prop/direct/P69'), ('labelProperty', 'educated at'), ('objectsClasses', 'http://www.wikidata.org/entity/Q15936437,http://www.wikidata.org/entity/Q23002054,http://www.wikidata.org/entity/Q3918,http://www.wikidata.org/entity/Q902104,http://www.wikidata.org/entity/Q1188663'), ('labelsObjectsClasses', 'research university,private not-for-profit educational institution,university,private university,Colonial Colleges')]
[('property', 'http://www.wik

### Get the first 50 "television series" (wd:Q5398426) that are connected with "award received" (wd:P166)

In [27]:
queryString = """
SELECT DISTINCT ?televisionSerie ?labelTelevisionSerie ?award ?labelAward
WHERE 
{   
   ?televisionSerie wdt:P31 wd:Q5398426 ;
            wdt:P166 ?award .
   ?televisionSerie sc:name ?labelTelevisionSerie .
   ?award sc:name ?labelAward .
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('televisionSerie', 'http://www.wikidata.org/entity/Q1758643'), ('labelTelevisionSerie', 'Nytt på nytt'), ('award', 'http://www.wikidata.org/entity/Q12715196'), ('labelAward', 'Gullruten for best comedy')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q265519'), ('labelTelevisionSerie', 'heute-show'), ('award', 'http://www.wikidata.org/entity/Q185578'), ('labelAward', 'German Comedy Awards')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q265519'), ('labelTelevisionSerie', 'heute-show'), ('award', 'http://www.wikidata.org/entity/Q472379'), ('labelAward', 'Grimme-Preis')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q265519'), ('labelTelevisionSerie', 'heute-show'), ('award', 'http://www.wikidata.org/entity/Q678413'), ('labelAward', 'Deutscher Fernsehpreis')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q691528'), ('labelTelevisionSerie', 'Wallace and Gromit'), ('award', 'http://www.wikidata.org/entity/Q838121'), ('labelAward', 'Peabody Awards')]
[

### Final Query

Final query for this task

In [11]:
queryString = """
#Not distinct on count because a tvSerie can win the same award more than one time in different years
SELECT DISTINCT ?televisionSerie ?labelTelevisionSerie (COUNT(?award) AS ?numberOfAwards)
{
    ?televisionSerie wdt:P161 ?castMember ;
                     wdt:P31 wd:Q5398426 ;
                     wdt:P166 ?award ;
                     sc:name ?labelTelevisionSerie .
    
    
    #Cast Members of Cold Case
    {
        SELECT DISTINCT ?castMember ?labelCastMember 
        WHERE 
        {   
            #Cast Members of wd:Q733960 
            {
                SELECT DISTINCT ?castMember ?labelCastMember
                WHERE
                {
                    wd:Q733960 wdt:P161 ?castMember .
                    ?castMember sc:name ?labelCastMember .
                }
            }    
            #Cast Members of seasons of wd:Q733960 
            UNION
            {
                SELECT DISTINCT ?castMember ?labelCastMember
                WHERE
                {
                    wd:Q733960 wdt:P527 ?season .
                    ?season wdt:P161 ?castMember .
                    ?castMember sc:name ?labelCastMember .
                }
            }       
            #Cast Members of episodes of wd:Q733960 
            UNION
            {
                SELECT DISTINCT ?castMember ?labelCastMember
                WHERE
                {
                    wd:Q733960 wdt:P527 ?season .
                    ?season wdt:P527 ?episode .
                    ?episode wdt:P161 ?castMember .
                    ?castMember sc:name ?labelCastMember .
                }
            }
            ?castMember wdt:P106 ?occupation .
            ?occupation sc:name ?labelOccupation .    
            FILTER (REGEX (?labelOccupation, "(?:^|\\\\W)actor(?:$|\\\\W)") || REGEX (?labelOccupation, "(?:^|\\\\W)actress(?:$|\\\\W)"))  .
        }
    }
}GROUP BY ?televisionSerie ?labelTelevisionSerie
HAVING (COUNT(?award) > 2)
"""
print("Results")
x=run_query(queryString)

Results
[('televisionSerie', 'http://www.wikidata.org/entity/Q564345'), ('labelTelevisionSerie', 'Person of Interest'), ('numberOfAwards', '8')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q585758'), ('labelTelevisionSerie', 'Boardwalk Empire'), ('numberOfAwards', '3')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q647001'), ('labelTelevisionSerie', 'Dr. Quinn, Medicine Woman'), ('numberOfAwards', '22')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q464865'), ('labelTelevisionSerie', 'The Nanny'), ('numberOfAwards', '8')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q207803'), ('labelTelevisionSerie', 'Bones'), ('numberOfAwards', '3')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q30599007'), ('labelTelevisionSerie', 'Succession'), ('numberOfAwards', '8')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q1247192'), ('labelTelevisionSerie', 'Judging Amy'), ('numberOfAwards', '3')]
[('televisionSerie', 'http://www.wikidata.org/entity/Q193462'